In [0]:
%tensorflow_version 2.x

import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, LeakyReLU
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras import optimizers

warnings.filterwarnings('ignore')
tf.config.list_physical_devices('GPU')

[]

In [0]:
def parser(x):
    return pd.datetime.strptime(x,'%m/%d/%Y')

df = pd.read_csv('/content/raw_pf2.csv', parse_dates=[0], date_parser=parser)
df.tail()

,Week,GO_price,GD_US,GD_EU,GD_Asia,GS_US,GS_EU,GS_Asia,RM_Asia,RM_US,RM_EU,RR_Asia,RR_US,RR_EU,RU_N_US,RU_LT_US,RU_EU,RU_Asia,GI_US,GI_ARA,GI_SGP
151,2018-11-30,14.042,7401,6839,9649,7503,5614,10786,1366,2325,1893,29256,22960,12527,0.32,0.37,0.45,0.37,125612,15156,10769
152,2018-12-07,12.326,7401,6839,9649,7663,5905,10492,1193,2032,789,29122,23212,13102,0.32,0.37,0.45,0.36,124137,14947,10616
153,2018-12-14,11.924,7401,6839,9649,7663,5905,10492,1193,2032,789,29122,23212,13102,0.32,0.37,0.45,0.36,119900,14187,10916
154,2018-12-21,11.546,7401,6839,9649,7663,5905,10492,1193,2032,789,29122,23212,13102,0.32,0.37,0.45,0.36,119902,15492,11112
155,2018-12-28,10.610,7401,6839,9649,7663,5905,10492,1193,2032,789,29122,23212,13102,0.32,0.37,0.45,0.36,129431,15350,11526


In [0]:
tar = df['GO_price'].values
tar = tar.reshape((-1,1))

split_percent = 0.85
split = int(split_percent*len(tar))

tar_train = tar[:split]

tar_test = tar[split:]

date_train = df['Week'][:split]
date_test = df['Week'][split:]

print(len(tar_train))
print(len(tar_test))

132
24


In [0]:
look_back = 6

train_generator = TimeseriesGenerator(tar_train, tar_train, length=look_back, batch_size=1)     
test_generator = TimeseriesGenerator(tar_test, tar_test, length=look_back, batch_size=1)

In [0]:
model = Sequential()

model.add(LSTM(units=200,activation=LeakyReLU(),input_shape=(look_back,1)))
model.add(Dense(1))
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='mse')

num_epochs = 70

model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/70
126/126 [==============================] - 1s 11ms/step - loss: 14.0986
Epoch 2/70
126/126 [==============================] - 1s 7ms/step - loss: 2.0546
Epoch 3/70
126/126 [==============================] - 1s 7ms/step - loss: 1.6269
Epoch 4/70
126/126 [==============================] - 1s 7ms/step - loss: 1.8493
Epoch 5/70
126/126 [==============================] - 1s 7ms/step - loss: 1.3028
Epoch 6/70
126/126 [==============================] - 1s 7ms/step - loss: 1.2096
Epoch 7/70
126/126 [==============================] - 1s 7ms/step - loss: 1.1101
Epoch 8/70
126/126 [==============================] - 1s 6ms/step - loss: 0.8572
Epoch 9/70
126/126 [==============================] - 1s 7ms/step - loss: 0.9561
Epoch 10/70
126/126 [==============================] - 1s 7ms/step - loss: 0.7381
Epoch 11/70
126/126 [==============================] - 1s 7ms/step - loss: 0.7764
Epoch 12/70
126/126 [==============================] - 1s 7ms/step - loss: 0.6521
Epoch 13/70
126/126 [==

In [0]:
import plotly.graph_objects as go

prediction = model.predict_generator(test_generator)

tar_train = tar_train.reshape((-1))
tar_test = tar_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = tar_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = tar_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "smth",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Price"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [0]:
tar = tar.reshape((-1))

def predict(num_prediction, model):
    prediction_list = tar[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Week'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1,freq='W-FRI').tolist()
    return prediction_dates

num_prediction = 12
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [0]:
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Predicted'
)
trace1 = go.Scatter(
    x = df['Week'],
    y = df['GO_price'],
    mode = 'lines',
    name = 'Old'
)

layout = go.Layout(
    title = "smth forecast",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Price"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()